# Coverage 백분위별 SE vs Energy AUROC 분석

QuCo-RAG 논문 기반 triplet extractor로 계산한 corpus coverage를 **상대적 백분위수**로 나누고,
각 분위에서 Semantic Entropy(SE)와 Semantic Energy의 AUROC를 비교합니다.

- 하위 20% → 20-40% → 40-60% → 60-80% → 상위 20%

In [4]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_auc_score

DATA_DIR = Path(".")

## 1. 데이터 로드

In [5]:
def load_data(dataset_name: str) -> list[dict]:
    path = DATA_DIR / f"{dataset_name}_with_corpus.json"
    with open(path) as f:
        data = json.load(f)
    return data["samples"]

truthfulqa_samples = load_data("truthfulqa")
halueval_samples = load_data("halueval")

print(f"TruthfulQA: {len(truthfulqa_samples)} samples")
print(f"HaluEval: {len(halueval_samples)} samples")

TruthfulQA: 200 samples
HaluEval: 200 samples


In [6]:
def get_coverage(sample: dict) -> float:
    if "corpus_stats" not in sample:
        return 0.0
    return sample["corpus_stats"].get("coverage", 0.0)

## 2. Coverage 분포 확인

In [7]:
tqa_coverages = [get_coverage(s) for s in truthfulqa_samples]
halu_coverages = [get_coverage(s) for s in halueval_samples]

fig = make_subplots(rows=1, cols=2, subplot_titles=("TruthfulQA", "HaluEval"))

fig.add_trace(
    go.Histogram(x=tqa_coverages, nbinsx=30, marker_color="#3498db"),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=halu_coverages, nbinsx=30, marker_color="#9b59b6"),
    row=1, col=2
)

fig.update_layout(height=350, showlegend=False, title_text="Corpus Coverage 분포")
fig.update_xaxes(title_text="Coverage")
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.show()

In [8]:
for name, coverages in [("TruthfulQA", tqa_coverages), ("HaluEval", halu_coverages)]:
    nonzero = [c for c in coverages if c > 0]
    print(f"\n{name}:")
    print(f"  전체: {len(coverages)}, Non-zero: {len(nonzero)} ({len(nonzero)/len(coverages)*100:.1f}%)")
    print(f"  Mean: {np.mean(coverages):.4f}, Median: {np.median(coverages):.4f}")


TruthfulQA:
  전체: 200, Non-zero: 136 (68.0%)
  Mean: 0.2777, Median: 0.1780

HaluEval:
  전체: 200, Non-zero: 104 (52.0%)
  Mean: 0.1135, Median: 0.0234


## 3. 5분위(Quintile) 분석

전체 데이터셋에서 coverage 기준 **상대적 백분위**로 5등분:
- 하위 20% / 20-40% / 40-60% / 60-80% / 상위 20%

In [9]:
def compute_auroc(labels: list[int], scores: list[float]) -> float | None:
    if len(set(labels)) < 2 or len(labels) < 5:
        return None
    try:
        return float(roc_auc_score(labels, scores))
    except:
        return None


QUINTILE_LABELS = ["하위 20%", "20-40%", "40-60%", "60-80%", "상위 20%"]


def analyze_by_quintile(samples: list[dict]):
    coverages = np.array([get_coverage(s) for s in samples])
    
    quintile_edges = np.percentile(coverages, [0, 20, 40, 60, 80, 100])
    
    results = []
    for i in range(5):
        low, high = quintile_edges[i], quintile_edges[i + 1]
        
        if i == 4:
            mask = (coverages >= low) & (coverages <= high)
        else:
            mask = (coverages >= low) & (coverages < high)
        
        bin_samples = [s for s, m in zip(samples, mask) if m]
        
        if len(bin_samples) < 5:
            results.append({
                "quintile": QUINTILE_LABELS[i],
                "percentile": f"{i*20}-{(i+1)*20}%",
                "coverage_low": low,
                "coverage_high": high,
                "n_samples": len(bin_samples),
                "n_hall": 0,
                "hall_rate": None,
                "se_auroc": None,
                "energy_auroc": None,
            })
            continue
        
        labels = [s["is_hallucination"] for s in bin_samples]
        se_scores = [s["semantic_entropy"] for s in bin_samples]
        energy_scores = [s["semantic_energy"] for s in bin_samples]
        
        results.append({
            "quintile": QUINTILE_LABELS[i],
            "percentile": f"{i*20}-{(i+1)*20}%",
            "coverage_low": low,
            "coverage_high": high,
            "n_samples": len(bin_samples),
            "n_hall": sum(labels),
            "hall_rate": sum(labels) / len(labels),
            "se_auroc": compute_auroc(labels, se_scores),
            "energy_auroc": compute_auroc(labels, energy_scores),
        })
    
    return pd.DataFrame(results)

In [10]:
tqa_df = analyze_by_quintile(truthfulqa_samples)
print("=== TruthfulQA 5분위 분석 ===")
tqa_df

=== TruthfulQA 5분위 분석 ===


,quintile,percentile,coverage_low,coverage_high,n_samples,n_hall,hall_rate,se_auroc,energy_auroc
0,하위 20%,0-20%,0.00000,0.000000,0,0,NaN,NaN,NaN
1,20-40%,20-40%,0.00000,0.106310,80,64,0.800000,0.638672,0.555664
2,40-60%,40-60%,0.10631,0.500000,37,30,0.810811,0.469048,0.309524
3,60-80%,60-80%,0.50000,0.500000,0,0,NaN,NaN,NaN
4,상위 20%,80-100%,0.50000,0.829332,83,71,0.855422,0.677817,0.679577


In [11]:
halu_df = analyze_by_quintile(halueval_samples)
print("=== HaluEval 5분위 분석 ===")
halu_df

=== HaluEval 5분위 분석 ===


,quintile,percentile,coverage_low,coverage_high,n_samples,n_hall,hall_rate,se_auroc,energy_auroc
0,하위 20%,0-20%,0.000000,0.000000,0,0,NaN,NaN,NaN
1,20-40%,20-40%,0.000000,0.000000,0,0,NaN,NaN,NaN
2,40-60%,40-60%,0.000000,0.054639,120,12,0.100000,0.458333,0.597994
3,60-80%,60-80%,0.054639,0.166667,39,1,0.025641,0.947368,0.947368
4,상위 20%,80-100%,0.166667,0.679969,41,8,0.195122,0.543561,0.598485


## 4. SE vs Energy AUROC 비교 그래프

In [12]:
def plot_quintile_auroc(df: pd.DataFrame, title: str):
    df_valid = df[df["se_auroc"].notna() & df["energy_auroc"].notna()].copy()
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name="Semantic Entropy",
        x=df_valid["quintile"],
        y=df_valid["se_auroc"],
        marker_color="#2ecc71",
        text=df_valid["se_auroc"].apply(lambda x: f"{x:.3f}"),
        textposition="outside",
    ))
    
    fig.add_trace(go.Bar(
        name="Semantic Energy",
        x=df_valid["quintile"],
        y=df_valid["energy_auroc"],
        marker_color="#e74c3c",
        text=df_valid["energy_auroc"].apply(lambda x: f"{x:.3f}"),
        textposition="outside",
    ))
    
    fig.add_hline(y=0.5, line_dash="dash", line_color="gray", annotation_text="Random")
    
    fig.update_layout(
        title=f"{title}: Coverage 백분위별 SE vs Energy AUROC",
        xaxis_title="Coverage 백분위",
        yaxis_title="AUROC",
        yaxis_range=[0, 1.15],
        barmode="group",
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    return fig

plot_quintile_auroc(tqa_df, "TruthfulQA").show()

In [13]:
plot_quintile_auroc(halu_df, "HaluEval").show()

## 5. 두 데이터셋 비교

In [14]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("TruthfulQA", "HaluEval"),
    horizontal_spacing=0.12
)

for col, (df, name) in enumerate([(tqa_df, "TruthfulQA"), (halu_df, "HaluEval")], 1):
    df_valid = df[df["se_auroc"].notna() & df["energy_auroc"].notna()].copy()
    
    fig.add_trace(
        go.Bar(name="SE", x=df_valid["quintile"], y=df_valid["se_auroc"],
               marker_color="#2ecc71", showlegend=(col==1),
               text=df_valid["se_auroc"].apply(lambda x: f"{x:.2f}"), textposition="outside"),
        row=1, col=col
    )
    fig.add_trace(
        go.Bar(name="Energy", x=df_valid["quintile"], y=df_valid["energy_auroc"],
               marker_color="#e74c3c", showlegend=(col==1),
               text=df_valid["energy_auroc"].apply(lambda x: f"{x:.2f}"), textposition="outside"),
        row=1, col=col
    )
    fig.add_hline(y=0.5, line_dash="dash", line_color="gray", row=1, col=col)

fig.update_layout(
    title="Coverage 백분위별 SE vs Energy AUROC",
    height=450,
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=1.08, xanchor="center", x=0.5)
)
fig.update_yaxes(title_text="AUROC", range=[0, 1.15], row=1, col=1)
fig.update_yaxes(range=[0, 1.15], row=1, col=2)
fig.update_xaxes(title_text="Coverage 백분위", row=1, col=1)
fig.update_xaxes(title_text="Coverage 백분위", row=1, col=2)

fig.show()

## 6. AUROC 차이 (SE - Energy) 추이

In [15]:
tqa_df["auroc_diff"] = tqa_df["se_auroc"] - tqa_df["energy_auroc"]
tqa_df["dataset"] = "TruthfulQA"

halu_df["auroc_diff"] = halu_df["se_auroc"] - halu_df["energy_auroc"]
halu_df["dataset"] = "HaluEval"

combined = pd.concat([tqa_df, halu_df], ignore_index=True)
combined = combined[combined["auroc_diff"].notna()]

fig = go.Figure()

for dataset, color in [("TruthfulQA", "#3498db"), ("HaluEval", "#9b59b6")]:
    df_sub = combined[combined["dataset"] == dataset]
    fig.add_trace(go.Scatter(
        x=df_sub["quintile"],
        y=df_sub["auroc_diff"],
        mode="lines+markers+text",
        name=dataset,
        marker=dict(size=12, color=color),
        line=dict(width=3, color=color),
        text=df_sub["auroc_diff"].apply(lambda x: f"{x:+.3f}"),
        textposition="top center",
    ))

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.add_hrect(y0=0, y1=0.3, fillcolor="green", opacity=0.1, line_width=0)
fig.add_hrect(y0=-0.3, y1=0, fillcolor="red", opacity=0.1, line_width=0)

fig.add_annotation(x="상위 20%", y=0.15, text="SE 우세", showarrow=False, font=dict(color="green", size=12))
fig.add_annotation(x="상위 20%", y=-0.15, text="Energy 우세", showarrow=False, font=dict(color="red", size=12))

fig.update_layout(
    title="Coverage 백분위별 AUROC 차이 (SE - Energy)",
    xaxis_title="Coverage 백분위 (낮음 → 높음)",
    yaxis_title="AUROC 차이 (SE - Energy)",
    yaxis_range=[-0.35, 0.35],
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()

## 7. 결과 요약

In [16]:
print("=" * 70)
print("Coverage 백분위별 SE vs Energy AUROC 요약")
print("=" * 70)

for dataset, df in [("TruthfulQA", tqa_df), ("HaluEval", halu_df)]:
    print(f"\n[{dataset}]")
    print(f"{'백분위':<12} {'n':>5} {'SE':>8} {'Energy':>8} {'차이':>8} {'우세'}")
    print("-" * 55)
    
    for _, row in df.iterrows():
        if row["se_auroc"] is None:
            print(f"{row['quintile']:<12} {row['n_samples']:>5} {'N/A':>8} {'N/A':>8} {'N/A':>8} N/A")
            continue
            
        diff = row["auroc_diff"]
        winner = "SE" if diff > 0.01 else ("Energy" if diff < -0.01 else "Tie")
        print(f"{row['quintile']:<12} {row['n_samples']:>5} {row['se_auroc']:>8.3f} {row['energy_auroc']:>8.3f} {diff:>+8.3f} {winner}")

Coverage 백분위별 SE vs Energy AUROC 요약

[TruthfulQA]
백분위              n       SE   Energy       차이 우세
-------------------------------------------------------
하위 20%           0      nan      nan     +nan Tie
20-40%          80    0.639    0.556   +0.083 SE
40-60%          37    0.469    0.310   +0.160 SE
60-80%           0      nan      nan     +nan Tie
상위 20%          83    0.678    0.680   -0.002 Tie

[HaluEval]
백분위              n       SE   Energy       차이 우세
-------------------------------------------------------
하위 20%           0      nan      nan     +nan Tie
20-40%           0      nan      nan     +nan Tie
40-60%         120    0.458    0.598   -0.140 Energy
60-80%          39    0.947    0.947   +0.000 Tie
상위 20%          41    0.544    0.598   -0.055 Energy
